In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
import os

In [5]:
path = '/content/drive/MyDrive/operaciones'
# Add the path to sys.path
sys.path.append(path)
os.chdir(path)
os.curdir

'.'

In [6]:
!pip install optuna

In [7]:
import optuna
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import shuffle
import joblib

In [8]:
### cargar bases_procesadas ####
x_train = joblib.load('/content/drive/MyDrive/operaciones/salidas/data_final/x_train.pkl')
y_train = joblib.load('/content/drive/MyDrive/operaciones/salidas/data_final/y_train.pkl')
x_test = joblib.load('/content/drive/MyDrive/operaciones/salidas/data_final/x_test.pkl')
y_test = joblib.load('/content/drive/MyDrive/operaciones/salidas/data_final/y_test.pkl')
x_val = joblib.load('/content/drive/MyDrive/operaciones/salidas/data_final/x_val.pkl')
y_val = joblib.load('/content/drive/MyDrive/operaciones/salidas/data_final/y_val.pkl')

In [9]:
#### Escalar ######################
x_train=x_train.astype('float32') ## para poder escalarlo
x_test=x_test.astype('float32') ## para poder escalarlo
x_val=x_val.astype('float32') ## para poder escalarlo
x_train.max()
x_train.min()
x_test.max()
x_test.min()
x_val.max()
x_val.min()

x_train /=255 ### escalarlo para que quede entre 0 y 1, con base en el valor máximo
x_test /=255
x_val /=255

In [10]:
###### verificar tamaños

x_train.shape
x_test.shape
x_val.shape

np.prod(x_train[1].shape) ## cantidad de variables por imagen

np.unique(y_train, return_counts=True)
np.unique(y_test, return_counts=True)

(array([0, 1, 2, 3]), array([154, 165, 200, 176]))

In [11]:
# Crear el generador de augmentación
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest' #se rellena los pixeles vacios con el pixel mas cercano
)

# Obtener clases y sus tamaños
classes, counts = np.unique(y_train, return_counts=True)
max_count = max(counts)

x_train_aug = []
y_train_aug = []

# Aumentar solo las clases que tengan menos de max_count
for c in classes:
    x_class = x_train[y_train == c]
    y_class = y_train[y_train == c]
    n_samples_needed = max_count - len(x_class)

    # Agregar los datos originales
    x_train_aug.extend(x_class)
    y_train_aug.extend(y_class)

    # Generar muestras augmentadas si es necesario
    if n_samples_needed > 0:
        gen = datagen.flow(x_class, y_class, batch_size=1)
        for _ in range(n_samples_needed):
            x_aug, y_aug = next(gen)
            x_train_aug.append(x_aug[0])
            y_train_aug.append(y_aug[0])

# Convertir a arrays numpy
x_train_balanced = np.array(x_train_aug)
y_train_balanced = np.array(y_train_aug)

# Barajar (opcional pero recomendable)
from sklearn.utils import shuffle
x_train_balanced, y_train_balanced = shuffle(x_train_balanced, y_train_balanced, random_state=42)

# Guardar si deseas
joblib.dump(x_train_balanced, '/content/drive/MyDrive/operaciones/salidas/data_final/x_train_balanced.pkl')
joblib.dump(y_train_balanced, '/content/drive/MyDrive/operaciones/salidas/data_final/y_train_balanced.pkl')

['/content/drive/MyDrive/operaciones/salidas/data_final/y_train_balanced.pkl']

In [12]:
###### verificar tamaños

np.unique(x_train_balanced, return_counts=True)
np.unique(y_train_balanced, return_counts=True)

(array([0, 1, 2, 3]), array([1600, 1600, 1600, 1600]))

In [13]:
# Asegúrate de que los datos estén barajados
x_train_balanced, y_train_balanced = shuffle(x_train_balanced, y_train_balanced, random_state=42)

# Input shape e info de clases
input_shape = x_train_balanced.shape[1:]  # (alto, ancho, canales)
num_classes = len(np.unique(y_train_balanced))  # En tu caso, 4

# Definir el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Entrenamiento
history = model.fit(
    x_train_balanced, y_train_balanced,
    validation_data=(x_val, y_val),
    epochs=5,
    batch_size=8,
    callbacks=[early_stop, checkpoint]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.4799 - loss: 3.4252

800/800 ━━━━━━━━━━━━━━━━━━━━ 424s 524ms/step - accuracy: 0.4799 - loss: 3.4231 - val_accuracy: 0.6110 - val_loss: 2.2807
Epoch 2/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 523ms/step - accuracy: 0.5426 - loss: 1.0251

800/800 ━━━━━━━━━━━━━━━━━━━━ 452s 536ms/step - accuracy: 0.5426 - loss: 1.0250 - val_accuracy: 0.6686 - val_loss: 0.7374
Epoch 3/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step - accuracy: 0.5952 - loss: 0.9448

800/800 ━━━━━━━━━━━━━━━━━━━━ 405s 507ms/step - accuracy: 0.5952 - loss: 0.9448 - val_accuracy: 0.7709 - val_loss: 0.5262
Epoch 4/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 459s 528ms/step - accuracy: 0.6145 - loss: 0.8633 - val_accuracy: 0.7911 - val_loss: 0.5734
Epoch 5/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 422s 527ms/step - accuracy: 0.6207 - loss: 0.8433 - val_accuracy: 0.7795 - val_loss: 0.7766
